# Approximate Nearest Neighbors:

# Image Recommendation System via Collaborative Filtering

# ***Please read the instructions very carefully***
This is a modified version of the previous question and requires you to use an artificial nearest neighbors library

We suggest you to use one of the following:
- [ScaNN](https://github.com/google-research/google-research/tree/master/scann)
- [FAISS](https://github.com/facebookresearch/faiss)
- [Annoy](https://github.com/spotify/annoy.git)

1.   Assignment must be implemented in Python 3 only.
2.   You are allowed to use libraries for data preprocessing (numpy, pandas, nltk etc) and for evaluation metrics, data visualization (matplotlib etc.).
3.   You will be evaluated not just on the overall performance of the model and also on the experimentation with hyper parameters, data prepossessing techniques etc.
4.   ⚠️ The Assignment will be evaluated automatically. Please adhere to taking proper inputs from `config.csv` file. You can change your `config.csv` file to experiment with your code. But at the end, make sure that your outputs are corresponding to input values in `config.csv`
5.   Strict plagiarism checking will be done. An F will be awarded for plagiarism.

## About the Dataset
Behance is a community art website where users showcase and discover creative work. Each user is able to “appreciate” (equivalent to a “like” on Instagram or a “react” on Facebook) an image, indicating that they like the image. It is in the website’s best interests to show users pictures that they would like, to keep them engaged for longer. For this question, given a set of pictures that a user has already appreciated, you have to show them a new picture that they would like based on what similar users appreciated.


<br><br>
**The dataset has information of 1 million appreciates of 63,497 users on 178,788 items. The file Behance appreciate 1M has a triplet in each line in the form of (user id, item id, unix timestamp).**

**Task: Take the inputs from the config.csv file and output the recommendations for a particular person**
- Collaborative Filtering is a way to predict items to the user based on the the
user’s history and the history of similar users. The similarity between users can be quantified by the number of images that both the users appreciated.
- The images appreciated by a similar user would be the most suitable images to show a user. Since we can find the similarity between any two users, we would be able to find the “nearest” neighbours of any user, allowing us to use a KNN-based algorithm to recommend new images to a user.
- Since people do not like seeing pictures that they have seen already. Make sure that you do not recommend pictures that a user has appreciated already.
- Output the final response will be saved in the file named ```config['output_file']```.


**Output file format:**
Populate the output file with images that the user has not seen of the k most
similar users, in descending order of their similarity. Each line in the output
file should be a duplet in the form of (item id, user id), where the user id is the
id of the kth similar user. The order of the images corresponding to the same
similar user would not matter. The output file would look something like this:
```
item_id_1_of_1st_similar_user 1st_most_similar_user_id
item_id_2_of_1st_similar_user 1st_most_similar_user_id
item_id_3_of_1st_similar_user 1st_most_similar_user_id
...
item_id_1_of_2nd_similar_user 2nd_most_similar_user_id
item_id_2_of_2nd_similar_user 2nd_most_similar_user_id
item_id_3_of_2nd_similar_user 2nd_most_similar_user_id
...
item_id_1_of_kth_similar_user kth_most_similar_user_id
item_id_2_of_kth_similar_user kth_most_similar_user_id
item_id_3_of_kth_similar_user kth_most_similar_user_id
```

You may use any other recommendation system that you wish to use. However,
evaluation script will score your submission by measuring the similarity between
users with the number of common images they appreciated.
The dataset was extracted using Behance’s API as a part of the paper
“Vista: A visually, socially, and temporally-aware model for artistic
recommendation, RecSys, 2016”. Check out this [Google Drive folder](https://drive.google.com/drive/folders/0B9Ck8jw-TZUEc3NlMjVXdDlPU1k?resourcekey=0-6_8ykn0o4fLc5fuTEm91xA) for
more information about the dataset.


Have fun! The users are waiting to see new pictures!

# CODE

### Import necessary libraries

In [2]:
!pip3 install scann


In [3]:
import time
import scann
import pickle
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix,lil_matrix
from sklearn.decomposition import TruncatedSVD


2022-01-27 19:40:34.673252: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-27 19:40:34.673307: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
# Config Generation Sample Code.
# ⚠️ Only for experimentation on your side.
# ⚠️ Should be commented during submission.

# df = pd.DataFrame(data=[{'id':276633,
#                   'k':5,
#                   'dataset_file':'./Behance_appreciate_1M',
#                   'output_file':'./output.txt'}])
# df.to_csv('config.csv')

In [5]:
config = pd.read_csv('config.csv').iloc[3]

In [6]:
config

Unnamed: 0                                    0
id                                       276633
k                                             5
dataset_file            ./Behance_appreciate_1M
output_file     ./testing/output_test_scann.txt
Name: 3, dtype: object

In [7]:
user = int(config['id'])
k_value = int(config['k'])+1

### Read the Data

In [8]:
with open(config['dataset_file'], 'r') as inFile:
    appreciate_data = inFile.readlines()

### Initialize a dictionary to store the item_ids that a user likes

### Go through each line of the input file and construct the user_likes dictionary

In [9]:
user_likes = dict()
for line in appreciate_data:
    line = line.strip()
    user_id = int(line.split()[0])
    item_id = int(line.split()[1])
    if user_id not in user_likes:
        user_likes[user_id] = list()
    user_likes[user_id].append(item_id)

In [10]:
lis1 = []
lis2 = []

for line in appreciate_data:
    line = line.strip()
    user_id = int(line.split()[0])
    item_id = int(line.split()[1]) 
    lis1.append(user_id)
    lis2.append(item_id)

lis1=list(set(lis1))
lis2=list(set(lis2))

In [11]:
user_idx={}
arc=0
for i in lis1:
    user_idx[i]=arc
    arc+=1
item_idx={}
arc=0
for i in lis2:
    item_idx[i]=arc
    arc+=1
    
inv_user_idx={}
for idx in user_idx.keys():
    inv_user_idx[user_idx[idx]]=idx

In [12]:
data=lil_matrix((len(lis1),len(lis2)),dtype=float)

In [13]:
for line in appreciate_data:
    line = line.strip()
    user_id = int(line.split()[0])
    item_id = int(line.split()[1])
    data[user_idx[user_id],item_idx[item_id]]=1.0

In [14]:
del appreciate_data

### Use your choice of Approximate Nearest Neigbor after Collaborative Filtering to find nearest neighbors

In [ ]:
# built data with less dimension
tt=TruncatedSVD(n_components=3000)
X=tt.fit_transform(data)

In [ ]:
# export developed data
# with open('./testing/test_3000.pkl','wb') as f:
#     pickle.dump(X, f)


In [15]:
# import developed data
with open('./testing/test_3000.pkl','rb') as f:
    X = pickle.load(f)

In [16]:
def diff_list(list1, list2):
    """ returns difference of two lists """
    return [x for x in list1 if x not in list2]

In [17]:
# your code here

def neighbors(user,k_value,datasvd):
    """ returns an iterable object (like list or generator) """
    Y=[]
    Y.append(datasvd[user_idx[user]])
    searcher = scann.scann_ops_pybind.builder(datasvd, k_value, "dot_product").tree(num_leaves=2000, num_leaves_to_search=100, training_sample_size=250000).score_ah(2, anisotropic_quantization_threshold=0.2).reorder(100).build()
    neighbors, distances = searcher.search_batched(Y)
    scores={}
    for i in range(len(neighbors)):
        scores[neighbors[i]]=distances[i]
    neighbors, distances = searcher.search_batched(Y)
    k_n_n_list=[]
    knn_dis=[]
    for i in neighbors[0]:
        k_n_n_list.append(inv_user_idx[i])
    for i in distances[0]:
        knn_dis.append(i)
    return  knn_dis,k_n_n_list

In [18]:
start=time.time()
dis,id=neighbors(user,k_value,X)
end=time.time()
print(end-start)

In [ ]:
id

### Open the output file to write all the lines to the file

In [5]:
outFile = open("output_scann.txt", 'w')
for n_user in id[:k_value]:
    user_id = n_user
    if user_id!=user:
        for item_id in diff_list(user_likes[user_id],user_likes[user]):
            outFile.write(str(item_id) + ' ' + str(user_id) + '\n')
outFile.close()


3628520
666917
2492899
3313136


> #### Top 10 ids nearest to user_id 276633 (including itself)
> + distance_unit = [197.92722 , 41.0624 , 38.034836 , 37.896236 , 36.92703 , 34.978985 , 29.046488 , 29.002932 , 28.925209 , 27.971037]
> + user_id =[276633 , 3628520 , 666917 , 2492899 , 3313136 , 2585131 , 1238354 , 332645 , 1859320 , 2186317]
> + Runtime = 5minutes 34seconds

### Answer the following questions:

#### Q1. **Explain how your choice of library works**

+ used ScaNN library over Faiss and Annoy since it can handle roughly twice as many queries per second for a given accuracy represented in below graph
<img src="https://1.bp.blogspot.com/--mbMV8fQY28/XxsvbGL_l-I/AAAAAAAAGQ0/Br9B3XGnBa07barUxC4XTi8hSDxYzwAEgCLcBGAsYHQ/s640/image5.png" width="800px"/>
<br/>
+ working
    + The main idea behind ScaNN is the vector quantization schemes quantized database elements with the aim of higher average distance may actually result in superior MIPS accuracy over the aim of minimizing the average distance between each vector x and its quantized form x̃.  
    + Anisotropic vector quantization also allows ScaNN to better estimate inner products that are likely to be in the top-k MIPS results and therefore achieve higher accuracy.
    + In short the performance gains are achievable by optimizing algorithms for the end goal of improving search accuracy rather than an intermediate goal such as compression distortion.

#### Q2. **Compare your choice of library with vanilla KNN.**
***Hint: Include Time Complexity, and explain the tradeoff with recall***

*your solution here*

#### Q3. **Compare your choice of library with implementation of ScaNN, faiss and annoy.**
***Hint: Include Time Complexity, and explain the tradeoff with recall***

*your solution here*